In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 10 10:07:11 2021

@author: Marc-André, Van Binh
        
"""
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode,plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import datetime

from scipy.ndimage import gaussian_filter


DATA_DIR = os.path.join('..', 'data')

tiny = pd.read_csv(os.path.join(DATA_DIR, 'tidy_data.csv'))
tiny =tiny.dropna(subset = ["x_coordinate"]) #need to remove NaN


tiny['x_coordinate_adj'] = np.where(tiny["x_coordinate"]<0, -tiny["x_coordinate"], tiny["x_coordinate"] )
tiny['y_coordinate_adj'] = np.where(tiny["x_coordinate"]<0, -tiny["y_coordinate"]+42, tiny["y_coordinate"]+42 )

#year hard coded for now
def compute_league_avg(df, year=2017):
    
    season = int(str(year) + str(year+1))
    df_copy = df[df["season"] == season].copy()
    df_copy["coord_tuple"] = df_copy[["x_coordinate_adj","y_coordinate_adj"]].apply(tuple, axis=1)
    
    
    data_league= np.zeros((100,85))
    
    for i,j in df_copy['coord_tuple']:
        if  np.isnan(i) or np.isnan(j):
            pass
        else:
            data_league [int(i),int(j)] += 1
    
    
    # total hours in the season
    season_matchs_drop = df_copy.drop_duplicates(subset=['game_id'], keep='last') # use date to keep the same match with different date
    season_hours = 0
    for i, txt in enumerate (season_matchs_drop['game_time']):
        time = txt.split(':')
        hour_match = int(time[0]) / 60.0 + int(time[1]) / 3600.0
        season_hours += max(hour_match, 1.0)
    
    
    data_league = data_league/(season_hours * 2) #need to count each game twice since two team, need to replace with actual calculation of total game time
    
    #freq_dist = df_copy[["x_coordinate","y_coordinate"]].value_counts().reset_index(name="freq_league")
    #freq_dist["freq_league"] = freq_dist["freq_league"]/1271
    return data_league

#team and year hard coded for now
def compute_team_avg(df, year=2017, team = "Colorado Avalanche"):
    
    season = int(str(year) + str(year+1))

    df_copy = df[df["season"] == season].copy()
    df_copy2 = df_copy[df_copy['team']==team].copy()
    df_copy2["coord_tuple"] = df_copy2[["x_coordinate_adj","y_coordinate_adj"]].apply(tuple, axis=1)
    
    data_team = np.zeros((100,85))
    
    for i,j in df_copy2['coord_tuple']:
        if  np.isnan(i) or np.isnan(j):
            pass
        else:
            data_team[int(i),int(j)] += 1
    
    # count team hours
    # count match as home & away in the season, drop duplicate for detail match
    team_matchs = df_copy.loc[(df_copy["home_team"] == team) | (df_copy['away_team'] == team)]  
    team_matchs_drop = team_matchs.drop_duplicates(subset=['game_id'], keep='last') # use date to keep the same match with different date

    team_hours = 0
    for i, txt in enumerate (team_matchs_drop['game_time']):
        time = txt.split(':')
        hour_match = int(time[0]) / 60.0 + int(time[1]) / 3600.0
        team_hours += max(hour_match, 1.0)
    
    data_team = data_team/team_hours 
    
    return data_team

def all_season_team_avg(df, start_year = 2016, end_year = 2020, sigma=4):
    
    team_year = {}
    team_year_shoot = {}
    # start year = 2016 and end year = 2020
    for year in range(start_year, end_year + 1):
        
        team_year[str(year)] = []
        season = int(str(year) + str(year+1))

        df_copy = df[df["season"] == season].copy()
        
        # all teams have been presenting in the season by searching both in home_team and away_team
        all_home_match = df_copy.drop_duplicates(subset=['home_team'], keep='last') # 
        all_away_match = df_copy.drop_duplicates(subset=['away_team'], keep='last') # 
        team_year[str(year)] = np.array(all_home_match['home_team'])
        for _,team in enumerate (np.array(all_away_match['away_team'])):
            if team not in team_year[str(year)]:
                team_year[str(year)].append(team)
   
    for year in range(start_year, end_year + 1): 
        
        # create a dict for all years and all teams
        # each year includes many teams, each year, team include shoot_frequence array
        team_year_shoot[str(year)] = {}
        test_league = compute_league_avg(tiny, year)
        for team in team_year[str(year)]:
            
            test_team = compute_team_avg(tiny, year, team)

            test_total = test_team - test_league
            test_total_fine = gaussian_filter(test_total, sigma=sigma) #smoothing results
            
            team_year_shoot[str(year)][team]  = test_total_fine
        
    return team_year_shoot
        
def plot_contourf(tiny, start_year = 2016, end_year = 2020):  
    team_year_shoot = all_season_team_avg(tiny, start_year = start_year, end_year = end_year)    
    
    for year in range(start_year, end_year + 1):
        for team in team_year_shoot[str(year)].keys():
            print('the team name is = {} in the year = {}'.format(team, year))
            test_total = team_year_shoot[str(year)][team]
            
            
            xx, yy = np.mgrid[0:100:100j, -42:42:85j]
            
            fig = plt.figure()
            ax = fig.gca()
            ax.set_xlim(0, 100)
            ax.set_ylim(-42, 42)
            #plots
            cfset = ax.contourf(xx, yy, test_total, cmap='bwr')
            cset = ax.contour(xx, yy, test_total, colors='k')
            # Label plots
            ax.clabel(cset, inline=1, fontsize=10)
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            fig.colorbar(cfset)
            cfset.set_clim(-0.03,0.03)#need to find a way to centralise white = 0
            #cset.set_clim(-0.03,0.03)#need to find a way to centralise white = 0
            plt.show()



def contour(df):
    
    # initial data
    # Xlabel, Ylabel create
    xx, yy = np.mgrid[0:100:100j, -42:42:85j] 
    tiny_year_init = team_year_shoot[list(df.keys())[0]]
    tiny_team_init = tiny_year_init[list(tiny_year_init.keys())[0]]

    fig = go.Figure(data =
        go.Contour(
            x = xx[:,1],
            y = yy[1,:], 
            z = np.rot90(np.fliplr(tiny_team_init)),
            colorscale='RdBu',
            reversescale=True,
    
        ))
    
    fig.add_layout_image(
            dict(
                source="../../figures/nhl_rink.png",
                xref="x",
                yref="y",
                x=-100,
                y=45,
                sizex=200,
                sizey=90,
                sizing="stretch",
                opacity=0.5,
                layer="above")
    )
    
    updatemenu = []
    buttons = []
    buttons1 = []
    
    
    # button with one option for each dataframe
    for year in df.keys():
        buttons.append(dict(method='restyle',
                            label=year,
                            visible=True
                      ))
    for year in df.keys():
        for team in df[year].keys():
            buttons1.append(dict(method='restyle',
                                label=team,
                                visible=True,
                                args=['z',[np.rot90(np.fliplr(df[year][team]))]]
                                )
                          )
    
    # some adjustments to the updatemenus
    updatemenu = []
    your_menu = dict()
    updatemenu.append(your_menu)
    your_menu = dict()
    updatemenu.append(your_menu)
    
    updatemenu[0]['buttons'] = buttons
    updatemenu[0]['direction' ]     ="down"
    updatemenu[0]['pad'       ]     ={"r": 10, "t": 10}
    updatemenu[0]['showactive']     =True
    updatemenu[0]['x'         ]     =0.06
    updatemenu[0]['xanchor'   ]     ="left"
    updatemenu[0]['y'         ]     =1.15
    updatemenu[0]['yanchor'   ]     ="top"
    
    updatemenu[1]['buttons'] = buttons1
    updatemenu[1]['direction' ]     ="down"
    updatemenu[1]['pad'       ]     ={"r": 10, "t": 10}
    updatemenu[1]['showactive']     =True
    updatemenu[1]['x'         ]     =0.4
    updatemenu[1]['xanchor'   ]     ="left"
    updatemenu[1]['y'         ]     =1.15
    updatemenu[1]['yanchor'   ]     ="top"
    updatemenu1 = [ go.layout.Updatemenu(updatemenu[0]), go.layout.Updatemenu(updatemenu[1])]
    # add dropdown menus to the figure
    fig.update_layout(showlegend=False, updatemenus=updatemenu1)
    
    
    fig.update_layout(
        annotations=[
            go.layout.Annotation(text="year", x=0.01, xref="paper", y=1.1, yref="paper",
                                 align="left", showarrow=False),
            go.layout.Annotation(text="team", x=0.36, xref="paper", y=1.1,
                                 yref="paper", showarrow=False),
        ])
    
    fig.show()
    

# run function
#plot_contourf(tiny, start_year = 2016, end_year = 2016)
team_year_shoot = all_season_team_avg(tiny, start_year = 2016, end_year = 2020)
contour(team_year_shoot)